In [8]:
# Importa las librerias a usar
import pandas as pd
import re
import numpy as np
import funcionesw as fw
import datetime

importar arvhivos a procesar

In [9]:
# ubicacion de los archivos
patch1 = '../input/Kilometros en Servicio MIO - 05-Mar-2024.xlsx'# Se importa el archivo xlxs
patch2 =  '../input/REPORTE FINALIZADO 05_MARZ_2024.xlsx'
patch3 = '../input/JERARQUIAS.xlsx'

# Se importa el archivo xlxs
kmsmio = pd.read_excel(patch1, sheet_name='Programacion')
repofi = pd.read_excel(patch2, sheet_name='Programacion')
bitacora = pd.read_excel(patch2, sheet_name='Bitacora')
jerarquias = pd.read_excel(patch3, sheet_name='Hoja1')

# Se eliminan las filas y las columnas vacias
kmsmio = kmsmio.dropna(axis=0, how='all').dropna(axis=1, how='all')
repofi = repofi.dropna(axis=0, how='all').dropna(axis=1, how='all')
bitacora = bitacora.dropna(axis=0, how='all').dropna(axis=1, how='all')

# Se crea el df de adicionales
adi = kmsmio[kmsmio['Kilometros Programados Plan'].isna()]

In [10]:
# Se seleccionan las columnas de adicional para el merge
adi = adi[['Número de Vehículo','Desde','hasta', 'Línea','Punto de inicio','Punto de término']]

# Convertir la columna 'Desde' a tipo datetime
adi['Desde'] = pd.to_datetime(adi['Desde'], format='%H:%M:%S')
adi['hasta'] = pd.to_datetime(adi['hasta'], format='%H:%M:%S')

# Restar un minuto a cada valor en la columna 'Desde'
adi['Desde_z'] = adi['Desde'] - pd.Timedelta(minutes=1)
adi['hasta_z'] = adi['hasta'] - pd.Timedelta(minutes=1)


# Se aplica nuevamente la estandarizacion en la hora
adi['Desde'] = adi['Desde'].apply(fw.extraer_hora_regex)
adi['hasta'] = adi['hasta'].apply(fw.extraer_hora_regex)

# Solo horas
adi['Desde_h'] = adi['Desde'].apply(fw.extraer_hora_regex_mod)
adi['hasta_h'] = adi['hasta'].apply(fw.extraer_hora_regex_mod)
# Solo horas
repofi['desde_h'] = repofi['desde'].apply(fw.extraer_hora_regex_mod)
repofi['hasta_h'] = repofi['hasta'].apply(fw.extraer_hora_regex_mod)

# Se aplica nuevamente la estandarizacion en la hora
adi['Desde_z'] = adi['Desde_z'].apply(fw.extraer_hora_regex)
adi['hasta_z'] = adi['hasta_z'].apply(fw.extraer_hora_regex)

# Crea una nueva columna llamada 'ID' que combine las columnas 'Número de secuencia de carro', 'Desde' y 'hasta_x'
repofi['idadi'] = repofi.apply(lambda row: f"{row['Número de secuencia de carro']}_{row['desde']}_{row['hasta']}", axis=1)
repofi['idy'] = repofi.apply(lambda row: f"{row['Número de secuencia de carro']}_{row['Línea']}_{row['Punto de inicio']}_{row['Punto de término']}_{row['desde_h']}_{row['hasta_h']}", axis=1)
adi['idadi'] = adi.apply(lambda row: f"{row['Número de Vehículo']}_{row['Desde']}_{row['hasta']}", axis=1)
adi['idadi2'] = adi.apply(lambda row: f"{row['Número de Vehículo']}_{row['Desde_z']}_{row['hasta_z']}", axis=1)
adi['idy'] = adi.apply(lambda row: f"{row['Número de Vehículo']}_{row['Línea']}_{row['Punto de inicio']}_{row['Punto de término']}_{row['Desde_h']}_{row['hasta_h']}", axis=1)


# # Se verifican si hay duplicados por id en el df adicionales
print(adi[adi.duplicated(subset='idadi', keep=False)])# Preguntar a cristan
# # Se verifican si hay duplicados por id en el df adicionales
print(adi[adi.duplicated(subset='idadi2', keep=False)])# Preguntar a cristan

      Número de Vehículo     Desde     hasta Línea Punto de inicio  \
20274              22117  16:11:09  16:40:28  A01A          SANBO2   
20275              22117  16:11:09  16:40:28  A01A          SANBO2   

      Punto de término   Desde_z   hasta_z Desde_h hasta_h  \
20274           SANBO2  16:10:09  16:39:28      16      16   
20275           SANBO2  16:10:09  16:39:28      16      16   

                         idadi                   idadi2  \
20274  22117_16:11:09_16:40:28  22117_16:10:09_16:39:28   
20275  22117_16:11:09_16:40:28  22117_16:10:09_16:39:28   

                                  idy  
20274  22117_A01A_SANBO2_SANBO2_16_16  
20275  22117_A01A_SANBO2_SANBO2_16_16  
      Número de Vehículo     Desde     hasta Línea Punto de inicio  \
20274              22117  16:11:09  16:40:28  A01A          SANBO2   
20275              22117  16:11:09  16:40:28  A01A          SANBO2   

      Punto de término   Desde_z   hasta_z Desde_h hasta_h  \
20274           SANBO2  16:10:0

In [11]:
adia = adi[['Número de Vehículo', 'idadi',]]
adib = adi[['Número de Vehículo', 'idadi2']]
adic = adi[['Número de Vehículo','idy']]

In [12]:
# Merge de mergebitacora y df adicional
merge_adi_repofi = pd.merge(repofi, adia, on='idadi', how='left')
# Merge de mergebitacora y df adicional
merge_adi_repof = pd.merge(merge_adi_repofi, adib,
                           left_on='idadi', right_on='idadi2', how='left')

In [13]:
merge_adi_repof.columns

Index(['Fecha', 'Día tipo', 'Designación de tarea vehículo', 'Línea',
       'Tipo de viaje corto', 'Sentido', 'Tipo de kilometros',
       'Número de secuencia de carro', 'Concesionario de Transporte',
       'Número de tarea vehículo', 'desde', 'hasta', 'Duración',
       'Punto de inicio', 'Punto de término', 'Largo', 'Número de viaje',
       'Tipo de vehículo del viaje', 'Secuencia de arcos',
       'Descripción Novedad', 'Subcontratista', 'Tipo de vehículo del viaje.1',
       'Incumplidos', 'Nro de caso', 'Tipo', 'Lugar', 'Jerarquia', 'Vehículo',
       'Kms Planeados', 'Kms Perdidos', 'Kms cancelados', 'desde_h', 'hasta_h',
       'idadi', 'idy', 'Número de Vehículo_x', 'Número de Vehículo_y',
       'idadi2'],
      dtype='object')

In [14]:
merge_ad_repof = pd.merge(merge_adi_repof, adic,
                        on='idy', how='left')

In [20]:
x = merge_ad_repof[(merge_ad_repof['Número de Vehículo_x'])|(merge_ad_repof['Número de Vehículo'])|(merge_ad_repof['Número de Vehículo'])]
x['Número de secuencia de carro'].value_counts()

Número de secuencia de carro
22151    15
22117    10
22043     6
53011     6
22098     5
23006     2
53008     2
54010     1
22017     1
21058     1
22029     1
22126     1
23045     1
Name: count, dtype: int64

In [22]:
merge_ad_repof[merge_ad_repof['Número de secuencia de carro']==53015]

,Fecha,Día tipo,Designación de tarea vehículo,Línea,Tipo de viaje corto,Sentido,Tipo de kilometros,Número de secuencia de carro,Concesionario de Transporte,Número de tarea vehículo,...,Kms Perdidos,Kms cancelados,desde_h,hasta_h,idadi,idy,Número de Vehículo_x,Número de Vehículo_y,idadi2,Número de Vehículo
297,2024-03-05,DHABIL5,11,A72A,Viaje de salida,NaN,Kms en Vacio,53015,BYNCPC,11,...,NaN,NaN,03,04,53015_03:54:57_04:44:57,53015_A72A_PCS-SAL_D51C14OE_03_04,NaN,NaN,NaN,NaN
298,2024-03-05,DHABIL5,11,A72A,Viaje comercial,Ida,Kms en Servicio,53015,BYNCPC,11,...,NaN,NaN,04,04,53015_04:45:12_04:54:31,53015_A72A_D51C14OE_T.C-A3_04_04,NaN,NaN,NaN,NaN
299,2024-03-05,DHABIL5,11,A72A,Viaje comercial,Ida,Kms en Servicio,53015,BYNCPC,11,...,NaN,NaN,04,05,53015_04:55:30_05:18:01,53015_A72A_T.C-A3_T.C-A3_04_05,NaN,NaN,NaN,NaN
300,2024-03-05,DHABIL5,11,A72A,Viaje comercial,Ida,Kms en Servicio,53015,BYNCPC,11,...,NaN,NaN,05,05,53015_05:19:00_05:41:31,53015_A72A_T.C-A3_T.C-A3_05_05,NaN,NaN,NaN,NaN
301,2024-03-05,DHABIL5,11,A72A,Viaje comercial,Ida,Kms en Servicio,53015,BYNCPC,11,...,NaN,NaN,05,06,53015_05:42:30_06:06:16,53015_A72A_T.C-A3_T.C-A3_05_06,NaN,NaN,NaN,NaN
302,2024-03-05,DHABIL5,11,A72A,Viaje comercial,Ida,Kms en Servicio,53015,BYNCPC,11,...,NaN,NaN,06,06,53015_06:08:30_06:36:05,53015_A72A_T.C-A3_T.C-A3_06_06,NaN,NaN,NaN,NaN
303,2024-03-05,DHABIL5,11,A72A,Viaje comercial,Ida,Kms en Servicio,53015,BYNCPC,11,...,NaN,NaN,06,07,53015_06:37:00_07:05:13,53015_A72A_T.C-A3_T.C-A3_06_07,NaN,NaN,NaN,NaN
304,2024-03-05,DHABIL5,11,A72A,Viaje comercial,Ida,Kms en Servicio,53015,BYNCPC,11,...,NaN,NaN,07,07,53015_07:06:30_07:36:05,53015_A72A_T.C-A3_T.C-A3_07_07,NaN,NaN,NaN,NaN
305,2024-03-05,DHABIL5,11,A72A,Viaje comercial,Ida,Kms en Servicio,53015,BYNCPC,11,...,NaN,NaN,07,08,53015_07:37:00_08:06:35,53015_A72A_T.C-A3_T.C-A3_07_08,NaN,NaN,NaN,NaN
306,2024-03-05,DHABIL5,11,A72A,Viaje comercial,Ida,Kms en Servicio,53015,BYNCPC,11,...,NaN,NaN,08,08,53015_08:07:45_08:34:20,53015_A72A_T.C-A3_T.C-A3_08_08,NaN,NaN,NaN,NaN


Porcesar K2

In [3]:
# Filtra el DF por Kms en Servicio
kmsmio = kmsmio[kmsmio['Tipo Kilómetros']== 'Kms en Servicio']
kmsmio = kmsmio[kmsmio['Kilometros Programados Plan'].notna()]

# Se aplica la funcion extraer_hora_regex para estandarizar la hora
kmsmio['Desde'] = kmsmio['Desde'].apply(fw.extraer_hora_regex)
kmsmio['hasta'] = kmsmio['hasta'].apply(fw.extraer_hora_regex)

# Se crea una nueva columna 'id'
kmsmio['id'] = kmsmio.apply(lambda row: '-'.join((str(row['Fecha']), str(row['Línea']), str(row['Número de tarea vehículo']), 
                                                   str(row['Desde']), str(row['hasta']))), axis=1)

# Se verifica que la columna id no tenga id repetidos
kmsmio['id'].value_counts().sort_values(ascending=False)


id
2024-03-05 00:00:00-A14A-1-05:00:00-05:12:10        1
2024-03-05 00:00:00-T47B-2-19:21:48-20:01:10        1
2024-03-05 00:00:00-A14A-1-05:26:00-05:38:10        1
2024-03-05 00:00:00-A14A-1-05:38:55-05:51:15        1
2024-03-05 00:00:00-A14A-1-05:52:00-06:04:43        1
                                                   ..
2024-03-05 00:00:00-A35A-99753-24:03:04-24:14:39    1
2024-03-05 00:00:00-A35A-99753-23:03:04-23:14:39    1
2024-03-05 00:00:00-A35A-99753-21:33:04-21:44:07    1
2024-03-05 00:00:00-A35A-99753-21:30:02-21:30:17    1
2024-03-05 00:00:00-A35A-99753-24:16:10-24:29:23    1
Name: count, Length: 18645, dtype: int64

Procesar reporte finalizado

In [4]:
# Se aplica la funcion extraer_hora_regex para estandarizar la hora
repofi['desde'] = repofi['desde'].apply(fw.extraer_hora_regex)
repofi['hasta'] = repofi['hasta'].apply(fw.extraer_hora_regex)

# Filtra el DF por Kms en Servicio
repofi = repofi[repofi['Tipo de kilometros']== 'Kms en Servicio']

# Se crea una nueva columna 'id'
repofi['id'] = repofi.apply(lambda row: '-'.join((str(row['Fecha']), str(row['Línea']), str(row['Número de tarea vehículo']), 
                                                   str(row['desde']), str(row['hasta']))), axis=1)

# Se seleccionan las columnas del reporte finalizado
repofi = repofi[['id', 'Largo', 'Número de secuencia de carro', 'Número de tarea vehículo', 'Secuencia de arcos', 
                 'Número de viaje', 'Incumplidos','Descripción Novedad','Concesionario de Transporte', 'desde', 'hasta']]

# Se verifica que el id no tenga datos repetidos
repofi['id'].value_counts().sort_values(ascending=False)

id
2024-03-05 00:00:00-A14A-1-05:00:00-05:12:10        1
2024-03-05 00:00:00-T47B-2-19:21:48-20:01:10        1
2024-03-05 00:00:00-A14A-1-05:26:00-05:38:10        1
2024-03-05 00:00:00-A14A-1-05:38:55-05:51:15        1
2024-03-05 00:00:00-A14A-1-05:52:00-06:04:43        1
                                                   ..
2024-03-05 00:00:00-A35A-99753-24:03:04-24:14:39    1
2024-03-05 00:00:00-A35A-99753-23:03:04-23:14:39    1
2024-03-05 00:00:00-A35A-99753-21:33:04-21:44:07    1
2024-03-05 00:00:00-A35A-99753-21:30:02-21:30:17    1
2024-03-05 00:00:00-A35A-99753-24:16:10-24:29:23    1
Name: count, Length: 18645, dtype: int64

Se realiza el respectivo Merge con los dos df

In [5]:
# Se realiza el merge con outer en la columna id
merge = pd.merge(kmsmio, repofi, on='id', how='outer')

# Se verifica si hay duplicados o no coincidencias 
print(merge[merge.duplicated(subset='id')])

# Se verifica que el numero de filas en los df sea igual (True), para poder concluir que el merge fue exitoso
print(f'Len == en los 3 df: ', merge.shape[0] == kmsmio.shape[0] == repofi.shape[0])

Empty DataFrame
Columns: [Fecha, Día tipo, Designación de tarea vehículo, Línea, Tipo de viaje corto, Sentido, Tipo Kilómetros, Número de Vehículo, Concesionario de Transporte_x, Número de tarea vehículo_x, Desde, hasta_x, Duración, Punto de inicio, Punto de término, Largo_x, Número de viaje_x, Tipo de vehículo del viaje, Secuencia de arcos_x, Descripción Novedad_x, Concesionario Programado, Tipo de vehículo Programado, Incumplidos_x, Kilometros Programados-Desvios, Kilometros Programados Plan, Clase de viaje, a evaluar, TAREA, LINEA, VIAJE, VEHICULO SGCO, Kms en Servicio (MTS PLANEADOS), PARADAS PLANEADAS, PARADAS REALIZADAS SGCO, % PARADAS REALIZADAS SGCO, Kms en Servicio (MTS SGCO), FUENTE SGCO, HORA REAL SALIDA, HORA REAL LLEGADA, id, Largo_y, Número de secuencia de carro, Número de tarea vehículo_y, Secuencia de arcos_y, Número de viaje_y, Incumplidos_y, Descripción Novedad_y, Concesionario de Transporte_y, desde, hasta_y]
Index: []

[0 rows x 50 columns]
Len == en los 3 df:  True

Calificaciones de jerarquias en Bitacora

In [6]:
# Selecciona las columnas a usar
jerarquias = jerarquias[['JERARQUIA HGI RECO2',	'CALIFICACION']]

# Esto elimina los espacios en blanco adelante y atras
jerarquias['JERARQUIA HGI RECO2'] = jerarquias['JERARQUIA HGI RECO2'].str.strip()

#combina los el df de bitacora con el de jerarquias
bitacora = pd.merge(bitacora, jerarquias, left_on='Jerarquía', right_on='JERARQUIA HGI RECO2', how='left' )

# Esta linea es para verificar las filas que no se cruzaron por jerarquia y agregarlas a la matriz fe jererquia en input
x = bitacora[(bitacora['Jerarquía'].str.startswith('Salida'))& bitacora['CALIFICACION'].isnull()]
x['Jerarquía'].value_counts()

Series([], Name: count, dtype: int64)

In [7]:
# Este codigo elimina los duplicado de la columna calificaciones eliminando el que no afecta a kilometros

# Convertir la columna 'CALIFICACION' a numérica para poder comparar y encontrar el máximo
bitacora['CALIFICACION'] = pd.to_numeric(bitacora['CALIFICACION'], errors='coerce')

# Agrupar por las columnas 'Viaje' y 'Tarea', y mantener el valor máximo de 'CALIFICACION'
bitacora_grouped = bitacora.groupby(['Viaje', 'Tarea']).agg({'CALIFICACION': 'max'}).reset_index()

# Identificar las filas duplicadas en la tabla original
duplicados = bitacora[bitacora.duplicated(subset=['Viaje', 'Tarea'], keep=False)]

# Mantener solo una fila para cada grupo de duplicados en 'Viaje' y 'Tarea'
bitacora_sin_duplicados = bitacora.drop_duplicates(subset=['Viaje', 'Tarea'])

# Fusionar los datos filtrados con los valores máximos de 'CALIFICACION'
bitacora_final = pd.merge(bitacora_sin_duplicados, bitacora_grouped, on=['Viaje', 'Tarea'])

# Eliminar cualquiera de las filas duplicadas si ambas tienen valores nulos en 'CALIFICACION'
bitacora = bitacora_final.drop_duplicates(subset=['Viaje', 'Tarea'], keep='last', ignore_index=True)

# Se elimina la columna CALIFICACION_X y se renombra la columna CALIFICACION_Y
bitacora.drop(columns='CALIFICACION_x', axis=1, inplace=True)
bitacora.rename(columns={'CALIFICACION_y': 'CALIFICACION'}, inplace=True)

# Se verifica que no hay duplicados
bitacora[bitacora.duplicated(subset=['Viaje', 'Tarea'], keep=False)]

,# Caso MC,Tipo novedad,Jerarquía,Tarea,Viaje,Parada,Operador,Línea,Vehículo,Hora,...,Descripción,Jerarquia,Tipo de Novedad,Novedad,Retoma/Salida,Unnamed: 16,Unnamed: 17,Unnamed: 18,JERARQUIA HGI RECO2,CALIFICACION


Merge con bitacora


In [8]:
# Se reemplan las filas nulas por ceros de las dos columnas para posteriormente carbiar el tipo de dato
bitacora[['Tarea', 'Viaje']] = bitacora[['Tarea', 'Viaje']].fillna(0).astype(int)

# Se selecionan las columnas de Bitacora que se van a usar
bitacora = bitacora[['Jerarquía',	'Tarea',	'Viaje', 'Parada', 'Vehículo', 'Novedad', 'CALIFICACION']]
# Se filtran los casos que no tiene numero de viaje ya que no tiene retoma
bitacora = bitacora[bitacora['Viaje'] != 0]

# Se agrega al merge anterior la informacion de bitacora
# Importante tener en cuenta los vehiculos que tengan mas de un caso en la bitacora, esto genera id duplicados.
mergebitacora = pd.merge(merge, bitacora, left_on=['Número de tarea vehículo_x', 'Número de viaje_x'], right_on=['Tarea', 'Viaje'], how='outer' )

# Revisar este resultado, ya que los valores que no coinceden deben ser iguales en el filtro
print(f'len == en los 2 df ',mergebitacora.shape[0] ==  merge.shape[0])

len == en los 2 df  True


In [9]:
# Elimina las filas que 'Número de Vehículo' y 'Número de secuencia de carro' esten vacios
def eliminar_no_realizados(row):
    if pd.isna(row['Número de Vehículo']) and pd.isna(row['Número de secuencia de carro']):
        index = row.name
        mergebitacora.drop(index, axis=0, inplace=True) 
# Aplicar la función a cada fila del DataFrame mergebitacora
mergebitacora.apply(eliminar_no_realizados, axis=1)


# Cambia Blanco y Negro Masivo por BYN en la columna Concesionario Programado
mergebitacora['Concesionario Programado'] = mergebitacora['Concesionario Programado'].replace('Blanco y Negro Masivo','BYN')

,Fecha,Día tipo,Designación de tarea vehículo,Línea,Tipo de viaje corto,Sentido,Tipo Kilómetros,Número de Vehículo,Concesionario de Transporte_x,Número de tarea vehículo_x,...,Concesionario de Transporte_y,desde,hasta_y,Jerarquía,Tarea,Viaje,Parada,Vehículo,Novedad,CALIFICACION
0,2024-03-05,DHABIL,1,A14A,Viaje comercial,Ida,Kms en Servicio,32013,ETM,1,...,NaN,05:00:00,05:12:10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-03-05,DHABIL,1,A14A,Viaje comercial,Vuelta,Kms en Servicio,32013,ETM,1,...,NaN,05:12:55,05:25:15,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-03-05,DHABIL,1,A14A,Viaje comercial,Ida,Kms en Servicio,32013,ETM,1,...,NaN,05:26:00,05:38:10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-03-05,DHABIL,1,A14A,Viaje comercial,Vuelta,Kms en Servicio,32013,ETM,1,...,NaN,05:38:55,05:51:15,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-03-05,DHABIL,1,A14A,Viaje comercial,Ida,Kms en Servicio,32013,ETM,1,...,NaN,05:52:00,06:04:43,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18640,2024-03-05,DHABIL,99753,A35A,Viaje de posicionamiento,NaN,Kms en Servicio,53009,BYNCPC,99753,...,BYNCPC,17:08:31,17:08:46,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18641,2024-03-05,DHABIL,99753,A35A,Viaje de posicionamiento,NaN,Kms en Servicio,53009,BYNCPC,99753,...,BYNCPC,16:36:17,16:36:32,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18642,2024-03-05,DHABIL,99753,A35A,Viaje de posicionamiento,NaN,Kms en Servicio,53009,BYNCPC,99753,...,BYNCPC,16:01:34,16:01:49,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18643,2024-03-05,DHABIL,99753,A35A,Viaje de posicionamiento,NaN,Kms en Servicio,53009,BYNCPC,99753,...,BYNCPC,15:31:34,15:31:49,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [225]:
# Condiciones para la columna condicional
condicion1 = mergebitacora['Largo_x'] != mergebitacora['Largo_y']
condicion2 = mergebitacora['Número de Vehículo'] != mergebitacora['Número de secuencia de carro']
condicion3 = mergebitacora['Secuencia de arcos_x'] != mergebitacora['Secuencia de arcos_y']
condicion4 = mergebitacora['Concesionario de Transporte_x'] != mergebitacora['Concesionario Programado']

# Aplicar condiciones y asignar valores a la nueva columna
mergebitacora['Condicional_ini'] = np.where(condicion1 & condicion2 & condicion3 & condicion4, 'Revisar largo, parada, vehículo y consecionario',
                                    np.where(condicion1 & condicion2 & condicion3, 'Revisar largo, parada y vehículo',
                                        np.where(condicion1 & condicion2 & condicion4, 'Revisar largo, vehiculo y consecionario',         
                                            np.where(condicion1 & condicion3 & condicion4, 'Revisar largo, parada y consecionario',
                                                np.where(condicion2 & condicion3 & condicion4, 'Revisar vehiculo, parada y consecionarios',        
                                                    np.where(condicion1 & condicion2, 'Revisar largo y vehiculo',
                                                        np.where(condicion1 & condicion3, 'Revisar largo y parada',
                                                            np.where(condicion1 & condicion4, 'Revisar largo y consecionario', 
                                                                np.where(condicion2 & condicion3, 'Revisar vehiculo y parada',
                                                                    np.where(condicion2 & condicion4, 'Revisar vehiculo y consecionario',        
                                                                        np.where(condicion3 & condicion4, 'Revisar parada y consecionario',                                            
                                                                            np.where(condicion1, 'Revisar largo', 
                                                                                np.where(condicion2, 'Revisar numero de vehiculo',
                                                                                    np.where(condicion3, 'Revisar paradas',
                                                                                        np.where(condicion4, 'Revisar consesionario', 'OK')))))))))))))))